# Queueing Model


You have to implement a queueing model. The model has 2 servers and 1 queue. Let’s say we
have server-1 and server-2.
The arrival of the customer: If both servers are idle, the customer will go to server-1. If any of the
servers are idle and the other one is busy then the customer will go to Idle sever. If both servers
are busy then the customer will stand in the queue.
The departure of a customer: Departure can occur from any of the servers. Then the next
customer from the queue will be chosen according to queue policy (FIFO, LIFO, or SJF). [See
the class lecture for solution hint]


Tasks:
1. Print clock_value, server_status of both servers, arrival list of queue, service times in
queue, next_arrival time, next_departure time of both servers, total delay,
area_under_q(t), area_under_b(t) for both servers at each clock step.
2. We have to implement SJF, LIFO. Report average_delay, expected number of customers
in the queue, expected utilization of the server-1, and server-2 for each queue policy.
3. Important: You cannot make separate files for each queue policy. You have to
submit just one .py file. You can input a variable in your code to choose the queue
policy. For example 1 for FIFO, 2 for SJF, and 3 for LIFO.
4. You can use the given Inter-arrivals and service times in your code.
Inter-arrival Times: A 1 = 0.4, A 2 = 1.2, A 3 = 0.5, A 4 =1.7, A 5 = 0.2, A 6 = 1.6, A 7 = 0.2, A 8 = 1.4, A 9 = 1.9, A10 = 0.7
Service times are: S 1 = 2.0, S 2 = 0.7, S3 = 0.2, S4 = 1.1, S5 = 3.7, S6 = 0.6
5. Then take interarrival and service times randomly from exponential distribution of mean
= 1.2 for inter-arrival times and mean = 1.3 for service times. You can use the
np.random.exponential(mean, size) function from python to do this job. ( use
np.random.seed(0) for consistent values
6. Prepare a report like below:
Take Interarrival times = exponential with mean 1.2
Take Service times = exponential with mean 1.3
Run the Simulation for 3 cases: num_delays until 10, 30, and 60 customers.

## Run code

In [ ]:
import numpy as np
import math

In [ ]:
class DSSQ:

    def __init__(self, n):
        np.random.seed(0) # seed 0
        self.interarrivals = list(np.random.exponential(1.2, n)) # exponential with mean 1.2
        self.service_times = list(np.random.exponential(1.3, n)) # exponential with mean 1.3
        # testing samples
        # self.interarrivals = [0.4, 1.2, 0.5, 1.7, 0.2, 1.6, 0.2, 1.4, 1.9, 0.7]
        # self.service_times = [2.0, 0.7, 0.2, 1.1, 3.7, 0.6, 1.2, 2.1, 0.5]

        self.clock = 0.0
        #for server 1
        self.next_arrival_time = self.interarrivals.pop(0) 
        self.next_departure_time = math.inf
        self.server_status = 0 #IDLE server 1

        # for server 2
        self.next_departure_time_2 = math.inf
        self.server_status_2 = 0 #IDLE server 2

        self.num_in_queue = 0

        self.times_of_arrival_in_queue=[] #Store arrival times of the customers in the queue
        self.service_times_in_queue=[]  #Store service times of the customers in the queue
         
        self.num_of_delay = 0 
        self.total_delay = 0

        # qt and bt part
        self.area_under_qt = 0.0
        self.area_under_bt = 0.0 # for server 1
        self.area_under_bt_2 = 0.0 # for server 2

        self.last_event_time = 0 #for calculate time diffrence
        self.last_server_status = 0 # for last server status1
        self.last_server_status_2 = 0 # for last server status2
        self.last_num_in_queue = 0 # for last number in queue

        #for the initial one. clock 0

        print("Clock: ", self.clock)
        print("Server 1 Status: ", self.server_status)
        print("Server 2 Status: ", self.server_status_2)

        print("Number of Queue: ", self.num_in_queue)
        print("Times of arrivals in Queue: ", self.times_of_arrival_in_queue)
        print("Service times in Queue: ", self.service_times_in_queue)

        print("Next Arrival Time: ", self.next_arrival_time)
        print("Next Departure Time of server 1: ", self.next_departure_time)
        print("Next Departure Time of server 2: ", self.next_departure_time_2)

        print("Number of Delays: ", self.num_of_delay)
        print("Total Delay: ", self.total_delay)

        print("Area under Q(t): ", self.area_under_qt)
        print("Area under B(t): ", self.area_under_bt)
        print("Area under B(t)2: ", self.area_under_bt_2)

        
        print("\n---------------------------------------------------------------\n")
    
    def start_simulation(self, n, queue_type):  #Start the simulation
        while self.num_of_delay<n: # simulate untill n cuntomer delay
            self.timing(queue_type)
        # performance meuser of server.
        average_delay = round((self.total_delay / n) , 4)
        expected_number_in_queue = round((self.area_under_qt / self.clock) , 4)
        expected_utilization_of_server_1 = round((self.area_under_bt / self.clock) , 4)
        expected_utilization_of_server_2 = round((self.area_under_bt_2 / self.clock) , 4)

        print("Performance Measure")
        print("Aerrage Delay: ", average_delay)
        print("Expected Number of Customers in the queue: ", expected_number_in_queue)
        print("Expected Utilization of the server-1: ", expected_utilization_of_server_1)
        print("Expected Utilization of the server-2: ", expected_utilization_of_server_2)
        print("\n----------------------------x-x-x-x-x-x-x----------------------------\n")

    
    def timing(self, queue_type): #Clock moves forward according to next event time (A/D)
        self.clock= min(self.next_arrival_time, self.next_departure_time, self.next_departure_time_2)

        if self.clock == self.next_arrival_time:
            self.arrival()
            print("Clock: ", self.clock, " (A)")
        elif self.clock == self.next_departure_time:
            self.departure(queue_type)
            print("Clock: ", self.clock, " (D1)")
        elif self.clock == self.next_departure_time_2:
            self.departure_2(queue_type)
            print("Clock: ", self.clock, " (D2)")
        
        self.update_register() #this will update qt and bt

        print("Server 1 Status: ", self.server_status)
        print("Server 2 Status: ", self.server_status_2)

        print("Number of Queue: ", self.num_in_queue)
        print("Times of arrivals in Queue: ", self.times_of_arrival_in_queue)
        print("Service times in Queue: ", self.service_times_in_queue)

        print("Next Arrival Time: ", self.next_arrival_time)
        print("Next Departure Time of server 1: ", self.next_departure_time)
        print("Next Departure Time of server 2: ", self.next_departure_time_2)

        print("Number of Delays: ", self.num_of_delay)
        print("Total Delay: ", self.total_delay)

        print("Area under Q(t): ", self.area_under_qt)
        print("Area under B(t): ", self.area_under_bt)
        print("Area under B(t)2: ", self.area_under_bt_2)
        
        print("\n---------------------------------------------------------------\n")


    def arrival(self): #Arrival algortihm
        self.next_arrival_time += self.interarrivals.pop(0)

        if self.server_status is 0: #server 1 is idle
            self.num_of_delay += 1
            self.server_status = 1
            self.next_departure_time = self.clock + self.service_times.pop(0)

        elif self.server_status_2 is 0: #server 2 is idle
            self.num_of_delay += 1
            self.server_status_2 = 1
            self.next_departure_time_2 = self.clock + self.service_times.pop(0)

        else : #server 1 and 2 is busy
            self.num_in_queue += 1
            self.times_of_arrival_in_queue.append(self.clock)#Store Arrival times of the cutomer in the queue
            self.service_times_in_queue.append(self.service_times.pop(0)) #Store service time of the customer in the queue


    def departure(self, queue_type): #Departure Algorithm
        if self.num_in_queue == 0: # queue is empty
            self.server_status = 0
            self.next_departure_time = math.inf
        
        else: # queue is not empty
            self.num_in_queue -= 1
            self.num_of_delay += 1

            #AS FIFO, pop first arrival and service time from the queue. 
            #IF LIFO we have to pop last arrival and service time
            #For SJF, finf the index of minimum service time from  service_times_in_queue list.
            #Then pop the arrival of that index from times_of_arrivalqueue for delay count and others.  
            
            if queue_type is "FIFO" :
                arrival = self.times_of_arrival_in_queue.pop(0) 
                self.next_departure_time = self.clock + self.service_times_in_queue.pop(0)
            
            elif queue_type is "LIFO" :
                arrival = self.times_of_arrival_in_queue.pop(-1) 
                self.next_departure_time = self.clock + self.service_times_in_queue.pop(-1)
            
            elif queue_type is "SJF" :
                min_index = self.service_times_in_queue.index(min(self.service_times_in_queue))
                arrival = self.times_of_arrival_in_queue.pop(min_index) 
                self.next_departure_time = self.clock + self.service_times_in_queue.pop(min_index)

            delay = self.clock - arrival
            self.total_delay += delay

    def departure_2(self, queue_type): #Departure Algorithm
        if self.num_in_queue == 0: # queue is empty
            self.server_status_2 = 0
            self.next_departure_time_2 = math.inf
        
        else: # queue is not empty
            self.num_in_queue -= 1
            self.num_of_delay += 1

            #AS FIFO, pop first arrival and service time from the queue. 
            #IF LIFO we have to pop last arrival and service time
            #For SJF, finf the index of minimum service time from  service_times_in_queue list.
            #Then pop the arrival of that index from times_of_arrivalqueue for delay count and others.  
            
            if queue_type is "FIFO" :
                arrival = self.times_of_arrival_in_queue.pop(0) 
                self.next_departure_time_2 = self.clock + self.service_times_in_queue.pop(0)
            
            elif queue_type is "LIFO" :
                arrival = self.times_of_arrival_in_queue.pop(-1) 
                self.next_departure_time_2 = self.clock + self.service_times_in_queue.pop(-1)
            
            elif queue_type is "SJF" :
                min_index = self.service_times_in_queue.index(min(self.service_times_in_queue))
                arrival = self.times_of_arrival_in_queue.pop(min_index) 
                self.next_departure_time_2 = self.clock + self.service_times_in_queue.pop(min_index)

            delay = self.clock - arrival
            self.total_delay += delay

    def update_register(self): #for qt and bt calculation.
        time_difference = self.clock - self.last_event_time
        self.area_under_qt +=  time_difference* self.last_num_in_queue
        self.area_under_bt +=  time_difference* self.last_server_status
        self.area_under_bt_2 +=  time_difference* self.last_server_status_2

        #update next event
        self.last_event_time = self.clock
        self.last_num_in_queue = self.num_in_queue
        self.last_server_status = self.server_status
        self.last_server_status_2 = self.server_status_2
        

In [ ]:
number_of_cases = [10, 30, 60]

for n in number_of_cases:
    print("N : ", n)

    print("\n------------------------FIFO-------------------------------\n")
    server = DSSQ(100)
    server.start_simulation(n, "FIFO")

    print("\n------------------------LIFO-------------------------------\n")
    server.__init__(100)
    server.start_simulation(n, "LIFO")

    print("\n------------------------SJF-------------------------------\n")
    server.__init__(100)
    server.start_simulation(n, "SJF")
    print("\n\n\n\n\n")

Streaming output truncated to the last 5000 lines.
Clock:  50.897680068048444  (A)
Server 1 Status:  1
Server 2 Status:  0
Number of Queue:  0
Times of arrivals in Queue:  []
Service times in Queue:  []
Next Arrival Time:  54.35117816253767
Next Departure Time of server 1:  52.27439228563277
Next Departure Time of server 2:  inf
Number of Delays:  38
Total Delay:  4.507506324836843
Area under Q(t):  4.507506324836843
Area under B(t):  30.149195154751606
Area under B(t)2:  20.305816671644216

---------------------------------------------------------------

Clock:  52.27439228563277  (D1)
Server 1 Status:  0
Server 2 Status:  0
Number of Queue:  0
Times of arrivals in Queue:  []
Service times in Queue:  []
Next Arrival Time:  54.35117816253767
Next Departure Time of server 1:  inf
Next Departure Time of server 2:  inf
Number of Delays:  38
Total Delay:  4.507506324836843
Area under Q(t):  4.507506324836843
Area under B(t):  31.525907372335933
Area under B(t)2:  20.305816671644216

------